In [3]:
%pip install --upgrade pip
%pip install numpy pandas scipy statsmodels matplotlib seaborn scikit-learn
%pip install gdown torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
# 1) Jeśli jeszcze nie masz zainstalowanego gdown, uruchom poniższą komórkę:
!pip install gdown --quiet

# 2) Importujemy gdown i pobieramy plik:
import gdown

# ID pliku wyciągnięte z linku "https://drive.google.com/file/d/1vJ18xuP32a6hDdwSUePXRNllPKRPZqN5/view"
file_id = "1vJ18xuP32a6hDdwSUePXRNllPKRPZqN5"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Ustawiamy nazwę, pod którą plik zostanie zapisany lokalnie
output_name = "zad2_wum_data_for_students.csv"

# Pobieramy plik do bieżącego katalogu
gdown.download(url, output_name, quiet=False)

# 3) Wczytujemy pobrany plik do pandas:
import pandas as pd

df = pd.read_csv(output_name, sep=";")
df.head()


Downloading...
From: https://drive.google.com/uc?export=download&id=1vJ18xuP32a6hDdwSUePXRNllPKRPZqN5
To: /Users/michalsmilowski/pythonProject14/zad2_wum_data_for_students.csv
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14.6M/14.6M [00:04<00:00, 3.51MB/s]


,Class,Output,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,...,Input391,Input392,Input393,Input394,Input395,Input396,Input397,Input398,Input399,Input400
0,0,0.800586,-0.002583,2.184037,-0.322008,1.621241,1.192444,-0.278356,-0.207366,0.735689,...,-2.140861,1.187660,0.345238,-0.844885,0.580007,-2.605781,-0.299471,0.711487,-0.753316,0.728763
1,0,2.168475,0.668637,1.373933,-0.476868,-0.724704,0.031162,-1.845921,0.784890,1.508526,...,-1.286120,-0.900044,-0.500399,-0.126421,-0.632233,-2.557419,0.056044,0.634774,-0.259835,0.106390
2,1,-1.210777,-0.681438,-0.544753,0.441346,-0.019906,-0.192135,-0.162510,-0.998777,0.686472,...,-0.391605,-0.190147,0.793746,-0.812737,-0.068228,-0.313143,2.564096,0.848355,0.180556,-1.525615
3,1,0.505678,-0.497957,0.720712,0.149120,0.019251,1.377850,0.981337,-0.846813,0.036790,...,-0.176734,-0.947351,-0.888601,1.509450,-0.501929,-0.554909,-0.104051,0.442150,-0.056644,1.447267
4,1,-10.281033,-1.178544,0.176941,1.112202,1.234189,0.999451,-0.773329,-0.811075,1.550537,...,-0.181325,0.198960,-0.697497,-0.836371,1.652071,0.974292,1.584071,-0.202352,1.362426,1.023857


**Baseline Models**

We start by defining simple baseline models to establish a “floor” performance that any more sophisticated method should exceed:

- **Linear Regression** for the continuous target (`Output`)  
- **Logistic Regression** for the categorical target (`Class`)

Both models are evaluated in two ways:  
1. **Train/Test Split** (80% training, 20% testing)  
2. **10-Fold Cross-Validation** (to gauge stability and generalization)

---

**Results**

- The **linear regression** model achieves an \(R^2\) of approximately **0.37**, indicating it explains only 37% of the variance in the continuous target—far from ideal.  
- The **logistic regression** model attains an **accuracy** of around **0.50**, which is barely better than random guessing in a binary or balanced multiclass scenario.

These poor scores confirm that simple linear methods are insufficient here and motivate the use of more advanced techniques (regularization, feature engineering, non-linear models, etc.).  


In [24]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model   import LinearRegression, LogisticRegression
from sklearn.metrics        import accuracy_score

# prepare data
X   = df.drop(['Output', 'Class'], axis=1)
y_r = df['Output']
y_c = df['Class']

# 1) Linear regression
X_r_train, X_r_test, y_r_train, y_r_test = train_test_split(
    X, y_r, test_size=0.2, random_state=42
)
model_reg = LinearRegression().fit(X_r_train, y_r_train)
r2 = model_reg.score(X_r_test, y_r_test)
print("Linear Regression R² on TEST data:", r2)

# Cross-validation
kf     = KFold(n_splits=10, shuffle=True, random_state=42)
r2_cv  = cross_val_score(LinearRegression(), X, y_r,
                         cv=kf, scoring='r2')
print("Linear Regression R² on CV folds:", r2_cv)
print("Mean CV R²:", r2_cv.mean())

print()

# 2) Logistic regression
X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(
    X, y_c, test_size=0.2, random_state=42
)
model_clf = LogisticRegression(max_iter=1000, random_state=42)
model_clf.fit(X_c_train, y_c_train)
y_c_pred = model_clf.predict(X_c_test)
acc = accuracy_score(y_c_test, y_c_pred)
print("Logistic Regression accuracy on TEST data:", acc)

# Cross-validation
acc_cv = cross_val_score(LogisticRegression(max_iter=1000, random_state=42),
                         X, y_c, cv=kf, scoring='accuracy')
print("Logistic Regression accuracy on CV folds:", acc_cv)
print("Mean CV accuracy:", acc_cv.mean())

Linear Regression R² on TEST data: 0.4463990746888591
Linear Regression R² on CV folds: [0.55195921 0.35236736 0.23694276 0.32884444 0.46170815 0.38280338
 0.15756861 0.43151613 0.39775886 0.45489592]
Mean CV R²: 0.37563648074364064

Logistic Regression accuracy on TEST data: 0.5175
Logistic Regression accuracy on CV folds: [0.495 0.475 0.485 0.505 0.545 0.555 0.525 0.52  0.51  0.485]
Mean CV accuracy: 0.51


### Detailed Model Descriptions

Below is a concise explanation of how each classifier operates and why it behaves the way it does on this dataset.

---

#### SVC Pipeline
This model first uses **SelectKBest** (univariate statistical test) to pick the 10 most discriminative features, then trains a **Support Vector Classifier** with an RBF (radial-basis) kernel.  
- **Selected features** (k = 10):  
  - Input2  
  - Input40  
  - Input41  
  - Input74  
  - Input238  
  - Input240  
  - Input293  
  - Input308  
  - Input330  
  - Input396  
- **Feature selection** reduces noise and focuses the SVM on the strongest signals.  
- The **RBF kernel** maps data into a high-dimensional space where classes can be separated by a hyperplane, handling nonlinearity.  
- Overall, this two-step approach balances dimensionality reduction with a powerful nonlinear classifier, yielding the best accuracy.


---

#### k-Nearest Neighbors (k-NN)
k-NN is a **lazy**, instance-based learner that predicts a new point’s class by majority vote among its k closest neighbors in feature space.  
- **Manhattan distance** measures absolute coordinate differences, which can be more robust when features vary on different scales.  
- Because k-NN makes no global model assumptions, its performance directly reflects the local structure of the data; here, moderate accuracy suggests some local clusters but also considerable overlap between classes.

---

#### SVM (RBF, C=1)
A standalone **RBF SVM** trains on all features without explicit feature selection.  
- The RBF kernel allows the model to carve out complex, curved decision boundaries.  
- The regularization parameter C balances margin width against misclassification: a lower C yields smoother (more general) boundaries.  
- Its relatively poor performance indicates that without prior feature filtering, the SVM overfits or struggles to ignore irrelevant dimensions.

---

#### Random Forest
An ensemble of decision trees, each trained on a bootstrap sample of the data and a random subset of features at each split.  
- **Bagging** (bootstrap aggregating) reduces variance by averaging many trees.  
- Random feature subsetting decorrelates individual trees, further stabilizing predictions.  
- Decision trees capture nonlinear interactions and feature thresholds, so the forest excels where linear methods fail.

---

#### Linear SVC
A linear-kernel SVM uses the same principle as logistic regression—finding a hyperplane that maximizes the margin between classes—but optimizes a hinge loss instead of likelihood.  
- It can be more robust to outliers than logistic regression.  
- However, like any linear method, it struggles when class structure is highly nonlinear.

---

#### Extra Trees
“Extremely randomized” trees follow the same ensemble principle as Random Forests but choose split thresholds at random rather than optimally.  
- This extra randomness often reduces variance and can speed up training.  
- The results suggest that random splits still capture enough structure to outperform pure linear models, though they trail the more systematically constructed Random Forest.

---

#### Gaussian Naïve Bayes
Assumes each feature is normally distributed and **conditionally independent** given the class.  
- Computes class probabilities via Bayes’ theorem under a diagonal covariance assumption.  
- Extremely fast to train, but the independence assumption is strong; correlated features or non-Gaussian distributions degrade performance.  

---

**Why the differences?**  
- **Linear models** (Logistic, Linear SVC) and **Naïve Bayes** rely on simple assumptions and often underfit complex data.  
- **k-NN** leverages local neighborhoods but suffers when classes overlap heavily.  
- **Kernel methods** (RBF SVM) and **trees** (Random Forest, Extra Trees) can learn nonlinear relationships, so they perform better when the decision boundary is not a straight line.  
- Combining **feature selection** with a flexible classifier (SVC Pipeline) often yields the best balance of generalization and expressivity.  


In [30]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    HistGradientBoostingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# 0) Load data
df = pd.read_csv(file_name, sep=";")
input_cols = [c for c in df.columns if c.lower().startswith("input")]
X = df[input_cols]
y = df['Class']

# 1) Define models
models = {
    "SVC Pipeline": Pipeline([
        ("select", SelectKBest(score_func=f_classif, k=10)),
        ("svc",    SVC(kernel="rbf", C=100.0, gamma="scale", random_state=42))
    ]),
    "k-NN (Manhattan, k=50)": KNeighborsClassifier(
        n_neighbors=50, metric="manhattan"
    ),
    "SVM (RBF, C=1)": SVC(kernel="rbf", C=1.0, gamma="scale", random_state=42),
    "RF (800 trees, max_depth=30)": RandomForestClassifier(
        n_estimators=800, max_depth=30, random_state=42
    ),
    "Linear SVC": Pipeline([
        ("scaler", StandardScaler()),
        ("lsvc",   LinearSVC(max_iter=5000, random_state=42))
    ]),
    "Extra Trees": ExtraTreesClassifier(
        n_estimators=200, max_depth=None, random_state=42
    ),
    "Gaussian NB": Pipeline([
        ("scaler", StandardScaler()),
        ("gnb",    GaussianNB())
    ])
}

# 2) 10-fold Stratified CV – only average metrics
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scoring = {
    'accuracy':  'accuracy',
    'precision': 'precision_macro',
    'recall':    'recall_macro',
    'f1':        'f1_macro'
}

results_cv = []
for name, model in models.items():
    cv_res = cross_validate(
        model, X, y, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False
    )
    results_cv.append({
        "Model":     name,
        "Accuracy":  np.mean(cv_res['test_accuracy']),
        "Precision": np.mean(cv_res['test_precision']),
        "Recall":    np.mean(cv_res['test_recall']),
        "F1-score":  np.mean(cv_res['test_f1'])
    })

metrics_cv_df = pd.DataFrame(results_cv).set_index("Model")
print("=== 10-fold CV – average metrics for each model ===")
print(metrics_cv_df)

# 3) 80/20 train-test split evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=41, stratify=y
)

for name, model in models.items():
    # Dopasuj model na zbiorze treningowym
    model.fit(X_train, y_train)
    
    if hasattr(model, "named_steps") and "select" in model.named_steps:
        selector = model.named_steps["select"]
        mask = selector.get_support()
        selected_feats = [feat for feat, use in zip(input_cols, mask) if use]
        
        print(f"\nModel: {name}")
        print("Wybrane cechy (SelectKBest, k=10):")
        for feat in selected_feats:
            print("  -", feat)
    
    y_pred = model.predict(X_test)
    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average="macro")
    rec  = recall_score(y_test, y_pred, average="macro")
    f1   = f1_score(y_test, y_pred, average="macro")
    
    print(f"  → Accuracy:  {acc:.4f}")
    print(f"    Precision: {prec:.4f}")
    print(f"    Recall:    {rec:.4f}")
    print(f"    F1-score:  {f1:.4f}")


=== 10-fold CV – average metrics for each model ===
                              Accuracy  Precision    Recall  F1-score
Model                                                                
SVC Pipeline                    0.8155   0.816163  0.815289  0.815305
k-NN (Manhattan, k=50)          0.6130   0.615445  0.612244  0.610479
SVM (RBF, C=1)                  0.5585   0.558515  0.558111  0.557468
RF (800 trees, max_depth=30)    0.6875   0.690017  0.686666  0.685825
Linear SVC                      0.5285   0.528459  0.528432  0.528134
Extra Trees                     0.6595   0.659998  0.659233  0.659044
Gaussian NB                     0.5870   0.587172  0.586922  0.586627

Model: SVC Pipeline
Wybrane cechy (SelectKBest, k=10):
  - Input2
  - Input40
  - Input41
  - Input74
  - Input238
  - Input240
  - Input293
  - Input308
  - Input330
  - Input396
  → Accuracy:  0.8075
    Precision: 0.8076
    Recall:    0.8076
    F1-score:  0.8075
  → Accuracy:  0.6175
    Precision: 0.6175
    R

### Detailed Regression Model Explanations

Below is a concise description of how each regression model works, followed by comments on its performance in both 10-fold cross-validation and an 80/20 train/test split.

---

#### Lasso Regression  
Lasso adds an L1 penalty to the ordinary least-squares objective, which encourages many coefficients to become exactly zero. This effectively performs feature selection by discarding less informative inputs and retaining only the strongest predictors.  
- **Behavior here**: Achieves the highest \(R^2\) of all models (~0.48 CV, ~0.53 test), and the lowest errors (MSE ~6.5 CV → ~5.9 test).  
- **Why it succeeds**: By zeroing out noisy or redundant features, it prevents overfitting in this high-dimensional setting.

---

#### Ridge Regression  
Ridge adds an L2 penalty (squared coefficient sum) to the loss. Rather than eliminating features, it shrinks all coefficients toward zero, reducing variance when features are correlated.  
- **Behavior here**: Moderate performance (CV \(R^2\) ~0.38, test ~0.45). Errors are higher than Lasso.  
- **Why it trails Lasso**: Shrinking but not removing unhelpful features leaves some noise in the model.

---

#### ElasticNet  
ElasticNet combines L1 and L2 penalties, blending the feature-selection effect of Lasso with the coefficient shrinkage of Ridge. The balance is controlled by the `l1_ratio`.  
- **Behavior here**: Very similar to Lasso (CV \(R^2\) ~0.48, test ~0.53) and slightly better MSE/MAE.  
- **Why it matches Lasso**: The chosen mix of L1/L2 leverages both selection and shrinkage, yielding robust estimates.

---

#### SVR Pipeline  
First applies univariate **SelectKBest** to pick the top 50 features by correlation with the target, then fits a **Support Vector Regressor** with an RBF kernel.  
- **Selected features (k = 50):**  
  - Input13  
  - Input18  
  - Input20  
  - Input25  
  - Input34  
  - Input35  
  - Input57  
  - Input83  
  - Input131  
  - Input136  
  - Input143  
  - Input153  
  - Input156  
  - Input167  
  - Input173  
  - Input179  
  - Input184  
  - Input193  
  - Input194  
  - Input195  
  - Input204  
  - Input205  
  - Input213  
  - Input217  
  - Input222  
  - Input223  
  - Input241  
  - Input250  
  - Input258  
  - Input270  
  - Input285  
  - Input286  
  - Input292  
  - Input294  
  - Input295  
  - Input298  
  - Input301  
  - Input302  
  - Input342  
  - Input345  
  - Input348  
  - Input359  
  - Input362  
  - Input368  
  - Input376  
  - Input377  
  - Input385  
  - Input388  
  - Input389  
  - Input390  
- **Behavior here**: Lower performance (CV \(R^2\) ~0.30, test ~0.34), higher errors (MSE ~8.7 CV → ~8.3 test).  
- **Why it underperforms**: Although nonlinear, SVR may still overfit or underfit if irrelevant features remain or the kernel parameters are not perfectly tuned. Even after removing 350 features, the remaining 50 may still include redundant or weak predictors.

---

#### k-Nearest Neighbors Regressor  
A nonparametric method that predicts each point’s output by averaging the targets of its k nearest neighbors in feature space (here, Euclidean distance).  
- **Behavior here**: Poorest results (CV \(R^2\) ~0.07, test ~0.12), and the largest errors (MSE ~11–11.7).  
- **Why it fails**: The high dimensionality and noise make “nearest” neighbors less meaningful, leading to unstable and biased predictions.

---

## RFECV + RidgeCV Pipeline

This pipeline first standardizes all inputs, then uses **RFECV** (recursive feature elimination with 5-fold CV on MSE) to peel away the least informative features (from 400 down to ~14), and finally fits a **Ridge** model whose α is chosen by **RidgeCV** (5-fold CV over `[0.01, 0.1, 1, 10, 100]`).

- **Selected features (RFECV eliminated down to ~14):**  
  - Input18  
  - Input59  
  - Input83  
  - Input136  
  - Input167  
  - Input173  
  - Input184  
  - Input193  
  - Input223  
  - Input241  
  - Input292  
  - Input342  
  - Input387  
  - Input389  

### Behavior here

- **Cross-validation**  
  - \(R^2 \approx 0.51\)  
  - MSE \(\approx 6.14\)  
  - MAE \(\approx 2.00\)

- **Hold-out (80/20 split)**  
  - \(R^2 \approx 0.53\)  
  - MSE \(\approx 5.68\)  
  - MAE \(\approx 1.93\)

### Why it excels

1. **Dimensionality reduction**  
   RFECV strips away noisy and redundant inputs so that the model sees only the most predictive ~14 features.  

2. **Optimized shrinkage**  
   RidgeCV finds the “just right” L2 penalty, taming multicollinearity without throwing away too much signal.  

3. **Synergy**  
   Combining targeted feature selection with tuned regularization yields a better bias–variance trade-off than either technique alone.

---

**Overall Takeaways**  
- The **RFECV + RidgeCV** pipeline now leads the pack in both CV and hold-out tests, surpassing even Lasso/ElasticNet by aggressively reducing noise before regularizing.  
- Sparse methods (Lasso/ElasticNet) remain strong baselines, but targeted feature elimination + tuned Ridge delivers the best generalization here.  
- Nonlinear approaches (SVR, k-NN) still lag unless paired with more sophisticated dimensionality reduction.  


In [31]:
import pandas as pd
import numpy as np

from sklearn.pipeline           import Pipeline
from sklearn.preprocessing     import StandardScaler
from sklearn.linear_model      import Lasso, Ridge, ElasticNet, RidgeCV
from sklearn.feature_selection  import RFECV, SelectKBest, f_regression
from sklearn.svm               import SVR
from sklearn.neighbors         import KNeighborsRegressor
from sklearn.model_selection   import KFold, train_test_split
from sklearn.metrics           import r2_score, mean_squared_error, mean_absolute_error

# 0) Load data
df = pd.read_csv(file_name, sep=";")
input_cols = [c for c in df.columns if c.lower().startswith("input")]
X = df[input_cols]
y = df['Output']

models = {
    "Lasso (alpha=0.114)": Pipeline([
        ("scaler", StandardScaler()),
        ("lasso", Lasso(alpha=0.114, max_iter=10000, random_state=42))
    ]),
    "Ridge (alpha=0.11)": Pipeline([
        ("scaler", StandardScaler()),
        ("ridge", Ridge(alpha=0.11, random_state=42))
    ]),
    "ElasticNet (l1_ratio=0.6)": Pipeline([
        ("scaler", StandardScaler()),
        ("en", ElasticNet(alpha=0.114, l1_ratio=0.6, max_iter=5000, random_state=42))
    ]),
    "SVR Pipeline": Pipeline([
        ("select", SelectKBest(score_func=f_regression, k=50)),
        ("svr", SVR(kernel="rbf", C=100.0, epsilon=0.1, gamma="scale"))
    ]),
    "k-NN Regressor (k=10)": KNeighborsRegressor(
        n_neighbors=10, metric="minkowski"
    ),
    "RFECV + RidgeCV": Pipeline([
        ("scaler", StandardScaler()),
        ("rfecv", RFECV(
            estimator=Ridge(random_state=42),
            step=1,
            cv=5,
            scoring="r2",
            n_jobs=-1
        )),
        ("ridgecv", RidgeCV(
            alphas=[0.01, 0.1, 1, 10, 100],
            cv=5,
            scoring="r2"
        ))
    ]),
}

# 1) 10-fold CV – average metrics (as before)
cv = KFold(n_splits=10, shuffle=True, random_state=42)
scoring = {
    'R2': 'r2',
    'MSE': 'neg_mean_squared_error',
    'MAE': 'neg_mean_absolute_error'
}

results_cv = []
for name, model in models.items():
    cv_res = cross_validate(
        model, X, y, cv=cv, scoring=scoring,
        n_jobs=-1, return_train_score=False
    )
    results_cv.append({
        "Model": name,
        "R2": np.mean(cv_res['test_R2']),
        "MSE": -np.mean(cv_res['test_MSE']),
        "MAE": -np.mean(cv_res['test_MAE'])
    })

metrics_cv_df = pd.DataFrame(results_cv).set_index("Model")
print("=== 10-fold CV – average metrics for each model ===")
print(metrics_cv_df)

# 2) 80/20 train-test split evaluation + print selected features (if any)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

for name, model in models.items():
    # Fit model on training data
    model.fit(X_train, y_train)
    
    # If the model has a SelectKBest step, print which features were selected
    if hasattr(model, "named_steps") and "select" in model.named_steps:
        selector = model.named_steps["select"]
        mask = selector.get_support()
        selected_feats = [feat for feat, use in zip(input_cols, mask) if use]
        
        print(f"\nModel: {name}")
        print("Selected features (SelectKBest, k=50):")
        for feat in selected_feats:
            print("  -", feat)
    
    # If the model has an RFECV step, print which features were selected
    if hasattr(model, "named_steps") and "rfecv" in model.named_steps:
        rfecv = model.named_steps["rfecv"]
        mask = rfecv.get_support()
        selected_feats = [feat for feat, use in zip(input_cols, mask) if use]
        
        print(f"\nModel: {name}")
        print("Selected features (RFECV):")
        for feat in selected_feats:
            print("  -", feat)
    
    # Compute and print test metrics
    y_pred = model.predict(X_test)
    r2  = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"  → R2:  {r2:.4f}")
    print(f"    MSE: {mse:.4f}")
    print(f"    MAE: {mae:.4f}")


=== 10-fold CV – average metrics for each model ===
                                 R2        MSE       MAE
Model                                                   
Lasso (alpha=0.114)        0.481601   6.490828  2.031663
Ridge (alpha=0.11)         0.375667   7.765346  2.217250
ElasticNet (l1_ratio=0.6)  0.478267   6.526011  2.036991
SVR Pipeline               0.302444   8.744197  2.372365
k-NN Regressor (k=10)      0.072118  11.736047  2.719089
RFECV + RidgeCV            0.505155   6.175246  1.998493
  → R2:  0.5316
    MSE: 5.9301
    MAE: 1.9694
  → R2:  0.4464
    MSE: 7.0078
    MAE: 2.0855
  → R2:  0.5343
    MSE: 5.8959
    MAE: 1.9614

Model: SVR Pipeline
Selected features (SelectKBest, k=50):
  - Input13
  - Input18
  - Input20
  - Input25
  - Input34
  - Input35
  - Input57
  - Input83
  - Input131
  - Input136
  - Input143
  - Input153
  - Input156
  - Input167
  - Input173
  - Input179
  - Input184
  - Input193
  - Input194
  - Input195
  - Input204
  - Input205
  - Input2

## Principal Component Analysis (PCA) Summary

1. **Original Data Shape**  
   - We start with a feature matrix **X** of shape **(2000, 400)**, meaning there are 2,000 samples and 400 raw input variables.

2. **Explained Variance by All Components**  
   - We fit PCA to **all 400 components** and compute the cumulative sum of explained variance ratios.  
   - This tells us, for each number of principal components, how much of the total variance in the original data is captured.

3. **Selecting Enough Components for 90% Variance**  
   - We set a threshold of **90%** cumulative explained variance.  
   - Using `np.searchsorted`, we find that **293** principal components are required to reach at least 90% of the total variance.

4. **Dimensionality Reduction**  
   - We then re-fit PCA with `n_components=293` and transform the original data.  
   - The resulting matrix **X_reduced** has shape **(2000, 293)**.

---

### What This Means

- **Dimensionality Reduction**: We have reduced the feature space from 400 dimensions down to 293 dimensions without losing more than 10% of the original variance.  
- **Trade-off**: By dropping 107 components (400 → 293), we are discarding only the least informative directions in the data, which likely correspond to noise or redundant information.  
- **Next Steps**: This lower-dimensional representation can speed up downstream modeling, reduce overfitting, and simplify feature selection, while still preserving the bulk of the signal present in the original inputs.  


In [27]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 2) Select only the input columns:
input_cols = [c for c in df.columns if c.lower().startswith("input")]
X = df[input_cols].values
print("Original X shape:", X.shape)

# 3) Fit PCA on all components:
pca_full = PCA().fit(X)
cumulative = np.cumsum(pca_full.explained_variance_ratio_)

# 4) Compute the number of components needed for 90% cumulative variance:
threshold = 0.90
k = np.searchsorted(cumulative, threshold) + 1
print(f"To achieve {int(threshold*100)}% cumulative variance, you need {k} components.")

# 5) Project onto those k components:
pca = PCA(n_components=k)
X_reduced = pca.fit_transform(X)
print("X_reduced shape:", X_reduced.shape)


Original X shape: (2000, 400)
To achieve 90% cumulative variance, you need 293 components.
X_reduced shape: (2000, 293)


### Impact of PCA on Model Performance

After reducing the original 400-dimensional feature space down to 293 principal components (capturing 90% of variance), we re-ran our top classification and regression pipelines. The results show a **significant drop in performance** compared to using the full feature set:

- **Classification (10-fold CV)**:  
  - SVC Pipeline: 81.6% → 65.9% accuracy  
  - k-NN (k=50):    61.3% → 54.4%  
  - RF (800, d=30): 68.8% → 57.7%  

- **Classification (80/20 split)**:  
  - SVC Pipeline: 80.8% → 65.8%  
  - k-NN:          61.8% → 54.5%  
  - RF:            71.0% → 53.3%  

- **Regression (10-fold CV)**:  
  - Lasso:    \(R^2\) 0.48 → 0.29, MSE ~6.5 → 8.9  
  - ElasticNet: 0.48 → 0.31, MSE ~6.5 → 8.7  

- **Regression (80/20 split)**:  
  - Lasso:    \(R^2\) 0.53 → 0.34, MSE ~5.9 → 8.4  
  - ElasticNet: 0.53 → 0.35, MSE ~5.9 → 8.2  

---

#### Why Performance Degrades with PCA

1. **Unsupervised reduction**  
   PCA captures directions of greatest variance **without regard to the target**. Variance in the data does not always align with the features most predictive of class labels or target values.  

2. **Loss of discriminative information**  
   By retaining components that explain overall variance, PCA may discard lower-variance dimensions that nonetheless carry critical signals for separation or regression.  

3. **Curse of dimensionality mitigation vs. task relevance**  
   While reducing dimension can help combat overfitting, here the retained 293 components still include many nuisance directions, and the reduction process itself removes subtle but important patterns.  

4. **Model sensitivity**  
   Classifiers like SVC and regressors like Lasso rely on specific feature relationships (e.g. marginal correlations) that PCA rotations obscure.

---

**Conclusion:**  
Although PCA preserves most of the raw variance, it is not optimized for predictive accuracy on our targets—especially when many small-variance directions are actually highly informative. For supervised tasks, techniques such as **supervised feature selection**, **Partial Least Squares (PLS)**, or **embedded methods** (e.g. Lasso feature selection) often outperform vanilla PCA.  


In [28]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    HistGradientBoostingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# 0) Load data
y = df['Class']

# 1) Define models
models = {
    "SVC Pipeline": Pipeline([
        ("select", SelectKBest(score_func=f_classif, k=5)),
        ("svc",    SVC(kernel="rbf", C=10.0, gamma="scale", random_state=42))
    ]),
    "k-NN (Manhattan, k=50)": KNeighborsClassifier(
        n_neighbors=50, metric="manhattan"
    ),
    "RF (800 trees, max_depth=30)": RandomForestClassifier(
        n_estimators=800, max_depth=30, random_state=42
    ),
}

# 2) 10-fold Stratified CV – only average metrics
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scoring = {
    'accuracy':  'accuracy',
    'precision': 'precision_macro',
    'recall':    'recall_macro',
    'f1':        'f1_macro'
}

results_cv = []
for name, model in models.items():
    cv_res = cross_validate(
        model, X_reduced, y, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False
    )
    results_cv.append({
        "Model":     name,
        "Accuracy":  np.mean(cv_res['test_accuracy']),
        "Precision": np.mean(cv_res['test_precision']),
        "Recall":    np.mean(cv_res['test_recall']),
        "F1-score":  np.mean(cv_res['test_f1'])
    })

metrics_cv_df = pd.DataFrame(results_cv).set_index("Model")
print("=== 10-fold CV – average metrics for each model ===")
print(metrics_cv_df)

# 3) 80/20 train-test split evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.2, random_state=41, stratify=y
)

results_test = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results_test.append({
        "Model":     name,
        "Accuracy":  accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average='macro'),
        "Recall":    recall_score(y_test, y_pred, average='macro'),
        "F1-score":  f1_score(y_test, y_pred, average='macro')
    })

metrics_test_df = pd.DataFrame(results_test).set_index("Model")
print("\n=== Test set results (80/20 split) ===")
print(metrics_test_df)


=== 10-fold CV – average metrics for each model ===
                              Accuracy  Precision    Recall  F1-score
Model                                                                
SVC Pipeline                    0.6595   0.661298  0.658971  0.658101
k-NN (Manhattan, k=50)          0.5485   0.549629  0.547184  0.542676
RF (800 trees, max_depth=30)    0.5710   0.571929  0.570049  0.567665

=== Test set results (80/20 split) ===
                              Accuracy  Precision    Recall  F1-score
Model                                                                
SVC Pipeline                    0.6425   0.642814  0.641932  0.641692
k-NN (Manhattan, k=50)          0.5600   0.560542  0.558701  0.555993
RF (800 trees, max_depth=30)    0.5550   0.554817  0.554225  0.553392


In [13]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_validate, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# 0) Load data
y = df['Output']

# 1) Define models
models = {
    "Lasso (alpha=0.114)": Pipeline([
        ("scaler", StandardScaler()),
        ("lasso", Lasso(alpha=0.114, max_iter=10000, random_state=42))
    ]),
    "ElasticNet (l1_ratio=0.6)": Pipeline([
        ("scaler", StandardScaler()),
        ("en", ElasticNet(alpha=0.114, l1_ratio=0.6, max_iter=5000, random_state=42))
    ]),
}

# 2) 10-fold CV – average metrics
cv = KFold(n_splits=10, shuffle=True, random_state=42)
scoring = {
    'R2': 'r2',
    'MSE': 'neg_mean_squared_error',
    'MAE': 'neg_mean_absolute_error'
}

results_cv = []
for name, model in models.items():
    cv_res = cross_validate(
        model, X_reduced, y, cv=cv, scoring=scoring,
        n_jobs=-1, return_train_score=False
    )
    results_cv.append({
        "Model": name,
        "R2": np.mean(cv_res['test_R2']),
        "MSE": -np.mean(cv_res['test_MSE']),
        "MAE": -np.mean(cv_res['test_MAE'])
    })

metrics_cv_df = pd.DataFrame(results_cv).set_index("Model")
print("=== 10-fold CV – average metrics for each model ===")
print(metrics_cv_df)

# 3) 80/20 train-test split evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42
)

results_tt = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results_tt.append({
        "Model": name,
        "R2": r2_score(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred)
    })

metrics_tt_df = pd.DataFrame(results_tt).set_index("Model")
print("\n=== 80/20 Train-Test Split – metrics for each model ===")
print(metrics_tt_df)

=== 10-fold CV – average metrics for each model ===
                                 R2       MSE       MAE
Model                                                  
Lasso (alpha=0.114)        0.304631  8.752247  2.354744
ElasticNet (l1_ratio=0.6)  0.325761  8.470025  2.315408

=== 80/20 Train-Test Split – metrics for each model ===
                                 R2       MSE       MAE
Model                                                  
Lasso (alpha=0.114)        0.351989  8.203404  2.269200
ElasticNet (l1_ratio=0.6)  0.373457  7.931644  2.222601


### Neural Network Model Summary

1. **Architecture**  
   - **Input layer**: 400 features  
   - **Hidden layers**:  
     - Dense(400 → 128) + ReLU  
     - Dense(128 → 64) + ReLU  
   - **Output layer**: Dense(64 → 1) for regression  

2. **Training Dynamics**  
   - Trained for 500 epochs with Adam (LR=1e-3, batch size 32).  
   - The **training loss (MSE)** started around 0.15, dipped below 0.02 by epoch 260, and stabilized around 0.02–0.04.  
   - Some fluctuation in loss indicates minor overfitting/noise, but the steady downward trend shows the network successfully learned patterns in the data.

3. **Final Test Performance**  
   - **Test MSE**: 7.53  
   - **Test R²**: 0.4055  

4. **Assessment**  
   - The network improves over a trivial constant predictor (which would have R²≤0), but its R² (~0.41) is still lower than our best linear baselines (Lasso/ElasticNet achieved ~0.53).  
   - This suggests that, despite fitting the training data well (low MSE), the model generalizes only moderately. Further regularization, architecture tuning, or more data might be needed to outperform simpler models.  


In [29]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

# 0) Hyperparameters
BATCH_SIZE = 32
EPOCHS = 500
LEARNING_RATE = 1e-3

# 1) Load data
df = pd.read_csv(file_name, sep=";")
input_cols = [c for c in df.columns if c.lower().startswith("input")]
X = df[input_cols].values.astype(np.float32)
y = df['Output'].values.reshape(-1, 1).astype(np.float32)

# 2) Split into train/test sets and standardize features
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3) Prepare DataLoaders
train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
X_test_t = torch.from_numpy(X_test)
y_test_t = torch.from_numpy(y_test)

# 4) Define the neural network model
class RegressionNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

# 5) Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegressionNN(X_train.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 6) Training loop
for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{EPOCHS} – Loss: {epoch_loss:.4f}")

# 7) Evaluation on the test set
model.eval()
with torch.no_grad():
    preds_test = model(X_test_t.to(device)).cpu().numpy()

mse = mean_squared_error(y_test, preds_test)
r2 = r2_score(y_test, preds_test)
print(f"\nTest MSE: {mse:.4f}")
print(f"Test R²: {r2:.4f}")


Epoch 10/500 – Loss: 0.1365
Epoch 20/500 – Loss: 0.1918
Epoch 30/500 – Loss: 0.1087
Epoch 40/500 – Loss: 0.1242
Epoch 50/500 – Loss: 0.0980
Epoch 60/500 – Loss: 0.0640
Epoch 70/500 – Loss: 0.1382
Epoch 80/500 – Loss: 0.0558
Epoch 90/500 – Loss: 0.0692
Epoch 100/500 – Loss: 0.0878
Epoch 110/500 – Loss: 0.0619
Epoch 120/500 – Loss: 0.0844
Epoch 130/500 – Loss: 0.0534
Epoch 140/500 – Loss: 0.0527
Epoch 150/500 – Loss: 0.0663
Epoch 160/500 – Loss: 0.0426
Epoch 170/500 – Loss: 0.0374
Epoch 180/500 – Loss: 0.0723
Epoch 190/500 – Loss: 0.0304
Epoch 200/500 – Loss: 0.0441
Epoch 210/500 – Loss: 0.0474
Epoch 220/500 – Loss: 0.0353
Epoch 230/500 – Loss: 0.0397
Epoch 240/500 – Loss: 0.0281
Epoch 250/500 – Loss: 0.0223
Epoch 260/500 – Loss: 0.0376
Epoch 270/500 – Loss: 0.0230
Epoch 280/500 – Loss: 0.0378
Epoch 290/500 – Loss: 0.0241
Epoch 300/500 – Loss: 0.0341
Epoch 310/500 – Loss: 0.0285
Epoch 320/500 – Loss: 0.0203
Epoch 330/500 – Loss: 0.0347
Epoch 340/500 – Loss: 0.0161
Epoch 350/500 – Loss: 0

In [38]:
import os
import pandas as pd
import numpy as np

from sklearn.pipeline           import Pipeline
from sklearn.preprocessing     import StandardScaler
from sklearn.feature_selection  import SelectKBest, f_classif, RFECV
from sklearn.svm               import SVC
from sklearn.linear_model      import Ridge, RidgeCV
from sklearn.model_selection   import train_test_split
from sklearn.metrics           import (
    r2_score, mean_squared_error, mean_absolute_error,
    accuracy_score, precision_score, recall_score, f1_score
)

def evaluate(file_path):
    """
    Reads a CSV from file_path (semicolon-delimited) and assumes:
      - columns starting with "Input" are features
      - column 'Output' is the regression target
      - column 'Class' is the classification target

    If file_path does not exist or cannot be read, prints an error message.

    For regression: builds the "RFECV + RidgeCV" pipeline, trains on an 80/20 split,
      and prints:
        - the features selected by RFECV
        - R2, MSE, MAE metrics on the test set

    For classification: builds the "SelectKBest + SVC" pipeline, trains on an 80/20 split
      (with stratify=y), and prints:
        - the features selected by SelectKBest
        - accuracy, precision, recall, F1 metrics on the test set
    """
    # Check if the file exists before attempting to load
    if not os.path.isfile(file_path):
        print(f"Error: The file path '{file_path}' is invalid or the file does not exist.")
        return

    try:
        # 1) Load data
        df = pd.read_csv(file_path, sep=";")
    except Exception as e:
        print(f"Error reading '{file_path}': {e}")
        return

    input_cols = [c for c in df.columns if c.lower().startswith("input")]
    X = df[input_cols]
    y_reg = df['Output']
    y_clf = df['Class']
    
    # 2) Split into training/test sets
    #    - regression (no stratify)
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X, y_reg, test_size=0.2, random_state=42
    )
    #    - classification (stratify=y_clf)
    X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
        X, y_clf, test_size=0.2, random_state=42, stratify=y_clf
    )
    
    # 3) Regression pipeline: RFECV + RidgeCV
    rfecv_ridge = Pipeline([
        ("scaler", StandardScaler()),
        ("rfecv", RFECV(
            estimator=Ridge(random_state=42),
            step=1,
            cv=5,
            scoring="r2",
            n_jobs=-1
        )),
        ("ridgecv", RidgeCV(
            alphas=[0.01, 0.1, 1, 10, 100],
            cv=5,
            scoring="r2"
        ))
    ])
    # Fit regression model
    rfecv_ridge.fit(X_train_reg, y_train_reg)
    
    # 3a) Extract features selected by RFECV
    rfecv_selector = rfecv_ridge.named_steps["rfecv"]
    mask_reg = rfecv_selector.get_support()
    selected_reg_feats = [feat for feat, use in zip(input_cols, mask_reg) if use]
    
    # 3b) Regression predictions and metrics on test set
    y_pred_reg = rfecv_ridge.predict(X_test_reg)
    r2 = r2_score(y_test_reg, y_pred_reg)
    mse = mean_squared_error(y_test_reg, y_pred_reg)
    mae = mean_absolute_error(y_test_reg, y_pred_reg)
    
    # 4) Classification pipeline: SelectKBest + SVC
    svc_pipeline = Pipeline([
        ("select", SelectKBest(score_func=f_classif, k=10)),
        ("svc",    SVC(kernel="rbf", C=100.0, gamma="scale", random_state=42))
    ])
    # Fit classification model
    svc_pipeline.fit(X_train_clf, y_train_clf)
    
    # 4a) Extract features selected by SelectKBest
    svc_selector = svc_pipeline.named_steps["select"]
    mask_clf = svc_selector.get_support()
    selected_clf_feats = [feat for feat, use in zip(input_cols, mask_clf) if use]
    
    # 4b) Classification predictions and metrics on test set
    y_pred_clf = svc_pipeline.predict(X_test_clf)
    acc  = accuracy_score(y_test_clf, y_pred_clf)
    prec = precision_score(y_test_clf, y_pred_clf, average="macro")
    rec  = recall_score(y_test_clf, y_pred_clf, average="macro")
    f1   = f1_score(y_test_clf, y_pred_clf, average="macro")
    
    # 5) Print results
    print("=== RFECV + RidgeCV (Regression) ===")
    print("Selected features (RFECV):")
    for feat in selected_reg_feats:
        print("  -", feat)
    print(f"\nRegression Metrics on Test Set:")
    print(f"  R2 : {r2:.4f}")
    print(f"  MSE: {mse:.4f}")
    print(f"  MAE: {mae:.4f}\n")
    
    print("=== SelectKBest + SVC (Classification) ===")
    print("Selected features (SelectKBest, k=10):")
    for feat in selected_clf_feats:
        print("  -", feat)
    print(f"\nClassification Metrics on Test Set:")
    print(f"  Accuracy : {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall   : {rec:.4f}")
    print(f"  F1-score : {f1:.4f}")

# Example usage:
evaluate("validation_data.csv")


Error: The file path 'validation_data.csv' is invalid or the file does not exist.
